## By Amir EL RIFAI and Mehdi CHENINI

# <center>Natural Language Processing Hands-on # 1</center>
<center><span style="font-weight: bold; font-size: 1.8rem;">Representing words and sentences</span></center>

Since most of the algorithms existing out there are designed to handle numerical data, they are hardly applicable on raw texts. However, it is definitely possible to convert a text to a numerical representation.

Ideal representations should handle **semantic**, **polysemy**, **irony** and lots of other specificities of texts. Along the decades, many text representations have been introduced to handle as many specificities as possible.

In this hands-on, you will have to convert a given corpus of texts to various representations and highlight their pros / cons.

# Installation of required packages

The packages listed below should be installed. Using a virtual environment is highly recommended but not mandatory -- that is just good practice.

In [ ]:
!pip install gensim

In [ ]:
import random

from pprint import pprint as pp

import gensim
import nltk
import numpy as np
import pandas as pd
import scipy as sp

from gensim.models import KeyedVectors
from gensim.test.utils import datapath
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import PCA

import nltk
nltk.download('reuters')
from nltk.corpus import reuters

START_TOKEN = '<START>'
END_TOKEN = '<END>'

np.random.seed(0)
random.seed(0)

[nltk_data] Downloading package reuters to /root/nltk_data...
[nltk_data]   Package reuters is already up-to-date!


**Note**: In NLP, we often add `<START>` and `<END>` tokens to represent the beginning and end of sentences, paragraphs or documents.

# Part 0 - Exploring the dataset

The Reuters Corpus that we will use contains 10,788 news documents totaling 1.3 million words. The documents have been classified into 90 categories.

Before diving into word representations, let's explore it a little bit and simply preprocess its texts to make it more suitable.

---

We will need to standardize all texts before converting anything to a numerical representations, since it will reduce the vocabulary size. Modify the following function to:

* Add the `START_TOKEN` and `END_TOKEN` at the beginning and end of each document
* Lowercase every words
* Remove the punctuation from each document

In [ ]:
def read_corpus(category="tea"):
    """ Read files from the specified Reuter's category.
        Params:
            category (string): category name
        Return:
            list of lists, with words from each of the processed files
    """
    files = reuters.fileids(category)

    listFiles = []
    for file in files:
        list = []
        list.append(START_TOKEN)
        for word in reuters.words(file):
            if word.isalpha() or word.isdigit():  # Remove punctuation
                list.append(word.lower())
        list.append(END_TOKEN)
        listFiles.append(list)

    return listFiles

**Purpose of the read_corpus function** :
The goal of the read_corpus function is to load text documents from the Reuters dataset and clean them in a consistent way so they are ready for later NLP steps.

The function takes a Reuters category as input here its 'tea'. It will only read documents that belong to this category.

The first thing it will do is tp retrieve the list of document identifiers corresponding to the chosen category.

We used a for loop so that for each document a new empty list will be created to store its words.

We added a special token at the beginnning of the document so that the model knows where the document starts.

For every word in the document:
	-	Punctuation is removed (isalpha() or isdigit())
	- Words are converted to lowercase to reduce vocabulary size
	- Cleaned words are added to the document list

At the end the cleaned document is added to the global list of documents.

This function will return  a list of documents, where each document is a list of normalized words and where all documents follow the same structure

In [ ]:
corpus = read_corpus()

In [ ]:
pp(corpus[:2], compact=True)

[['<START>', 'pakistan', 'confirms', 'kenya', 'tea', 'import', 'investigation',
  'pakistan', 's', 'corporate', 'law', 'authority', 'cla', 'has', 'begun', 'an',
  'enquiry', 'into', 'imports', 'of', 'tea', 'from', 'kenya', 'and', 'the',
  'trade', 'imbalance', 'between', 'the', 'two', 'countries', 'cla', 'chairman',
  'irtiza', 'husain', 'confirmed', 'he', 'told', 'reuters', 'by', 'telephone',
  'that', 'importers', 'liptons', 'and', 'brooke', 'bond', 'had', 'been',
  'asked', 'to', 'supply', 'data', 'to', 'the', 'authority', 'and', 'a',
  'hearing', 'would', 'be', 'held', 'the', 'cla', 'would', 'then', 'report',
  'back', 'to', 'the', 'commerce', 'ministry', 'which', 'had', 'requested',
  'the', 'enquiry', 'husain', 'said', 'no', 'date', 'had', 'yet', 'been', 'set',
  'for', 'the', 'hearing', 'and', 'declined', 'to', 'give', 'further',
  'details', 'of', 'the', 'matter', 'industry', 'sources', 'told', 'reuters',
  'reports', 'that', 'the', 'companies', 'tea', 'import', 'licences', 'ha

# Part I - Word representations

Now that we have preprocessed our texts we can represent them using vectors, also called embeddings in this case.

*Note: the preprocessing done here is basic. We will see in another hands-on different preprocessing steps, including some suitable for frequentist approaches.*

---

Each word representation has its pros and cons: understanding them will help you in finding the best representation that suits your use case.

As a result, you will have to implement / load and analyze the behaviour of word vectors coming from:

* Dummy encoding
* Co-occurence matrix encoding
* Pretrained GloVe encoding

## Dummy encoding

The dummy encoding consist in encoding each individual word of our corpus into a vector filled with 0 expect at a specific position where the value is 1 (equivalent to the encoding of categorical variables).

As discussed during the course, those embeddings are kind of pointless since they don't handle a single element of the ideal word representation beside being actual vectors. They are however a good starting point to play around with texts.

---

Define a function converting the words of a corpus to a set of dummy encoded vectors. Do not forget to sort your vocabulary before assigning the vectors!

In [ ]:
def dummy_encode_corpus(corpus):
    """
    Convert corpus words into dummy (one-hot) encoded vectors.

    Params:
        corpus: list of lists (documents made of tokens)

    Returns:
        vocab: sorted list of unique words
        word2dummy: dictionary {word: one-hot vector}
    """
    # 1. Build vocabulary
    vocab = sorted(set(word for doc in corpus for word in doc))

    # 2. Vocabulary size
    vocab_size = len(vocab)

    # 3. Create one-hot vectors
    word2dummy = {}
    for i, word in enumerate(vocab):
        vec = np.zeros(vocab_size)
        vec[i] = 1
        word2dummy[word] = vec

    return vocab, word2dummy

In [ ]:
vocab, word2dummy = dummy_encode_corpus(corpus)

print(vocab[:10])        # first 10 words alphabetically
print(word2dummy[vocab[0]])

['0', '000', '04', '075', '1', '10', '110', '123', '125', '134']
[1. 0. 0. ... 0. 0. 0.]


In [ ]:
type(corpus[0])

list

In [ ]:
def create_vector(i, size):
    vec = np.zeros(size)
    vec[i] = 1
    return vec

In [ ]:
create_vector(1, 20)

array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.])

In [ ]:
np.identity(3)[1]

array([0., 1., 0.])

In [ ]:
def dummy_encode(corpus):

    dummy = []

    for depeche in corpus:
        for word in depeche:
            dummy.append(word)

    dummy = list(set(dummy))

    vector_identity = np.identity(len(dummy))

    corpus_vector = []
    for depeche in corpus:
        depeche_vector = []
        for word in depeche:
            depeche_vector.append(vector_identity[dummy.index(word)])

        corpus_vector.append(depeche_vector)

    return corpus_vector

In [ ]:
dummy_encode(corpus)

[[array([0., 0., 0., ..., 1., 0., 0.]),
  array([0., 0., 0., ..., 0., 0., 0.]),
  array([0., 0., 0., ..., 0., 0., 0.]),
  array([0., 0., 0., ..., 0., 0., 0.]),
  array([0., 0., 0., ..., 0., 0., 0.]),
  array([0., 0., 0., ..., 0., 0., 0.]),
  array([0., 0., 0., ..., 0., 0., 0.]),
  array([0., 0., 0., ..., 0., 0., 0.]),
  array([0., 0., 0., ..., 0., 0., 0.]),
  array([0., 0., 0., ..., 0., 0., 0.]),
  array([0., 0., 0., ..., 0., 0., 0.]),
  array([0., 0., 0., ..., 0., 0., 0.]),
  array([0., 0., 0., ..., 0., 0., 0.]),
  array([0., 0., 0., ..., 0., 0., 0.]),
  array([0., 0., 0., ..., 0., 0., 0.]),
  array([0., 0., 0., ..., 0., 0., 0.]),
  array([0., 0., 0., ..., 0., 0., 0.]),
  array([0., 0., 0., ..., 0., 0., 0.]),
  array([0., 0., 0., ..., 0., 0., 0.]),
  array([0., 0., 0., ..., 0., 0., 0.]),
  array([0., 0., 0., ..., 0., 0., 0.]),
  array([0., 0., 0., ..., 0., 0., 0.]),
  array([0., 0., 0., ..., 0., 0., 0.]),
  array([0., 0., 0., ..., 0., 0., 0.]),
  array([0., 0., 0., ..., 0., 0., 0.]),


In [ ]:
dummy_embeddings = dummy_encode(corpus)

In [ ]:
w1 = vocab[0]
w2 = vocab[1]

np.dot(word2dummy[w1], word2dummy[w2])

np.float64(0.0)

In [ ]:
def cosine_similarity(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

In [ ]:
cosine_similarity(word2dummy[w1], word2dummy[w2])
cosine_similarity(word2dummy[w1], word2dummy[w1])

np.float64(1.0)

If you still do not believe that this representation is pointless, try finding the most similar word to "cat" using it.

## Co-occurence matrix encoding

*This section comes from Stanford's NLP hands-on*

A co-occurrence matrix counts how often things co-occur in some environment. Given some word  $w_i$  occurring in the document, we consider the context window surrounding  $w_i$ . Supposing our fixed window size is  $n$ , then this is the  $n$  preceding and  $n$  subsequent words in that document, i.e. words  $w_{i−n} … w_{i−1}$  and  $w_{i+1} … w{i+n}$ . We build a co-occurrence matrix  $M$ , which is a symmetric word-by-word matrix in which  $M_{ij}$  is the number of times  $w_j$  appears inside  $w_i$ 's window among all documents.

**Example: Co-Occurrence with Fixed Window of n=1:**

* Document 1: "all that glitters is not gold"

* Document 2: "all is well that ends well"

|        * 	| <START> 	| all 	| that 	| glitters 	| is 	| not 	| gold 	| well 	| ends 	| <END> 	|
|---------:	|--------:	|----:	|-----:	|---------:	|---:	|----:	|-----:	|-----:	|-----:	|------:	|
|  <START> 	|       0 	|   2 	|    0 	|        0 	|  0 	|   0 	|    0 	|    0 	|    0 	|     0 	|
|      all 	|       2 	|   0 	|    1 	|        0 	|  1 	|   0 	|    0 	|    0 	|    0 	|     0 	|
|     that 	|       0 	|   1 	|    0 	|        1 	|  0 	|   0 	|    0 	|    1 	|    1 	|     0 	|
| glitters 	|       0 	|   0 	|    1 	|        0 	|  1 	|   0 	|    0 	|    0 	|    0 	|     0 	|
|       is 	|       0 	|   1 	|    0 	|        1 	|  0 	|   1 	|    0 	|    1 	|    0 	|     0 	|
|      not 	|       0 	|   0 	|    0 	|        0 	|  1 	|   0 	|    1 	|    0 	|    0 	|     0 	|
|     gold 	|       0 	|   0 	|    0 	|        0 	|  0 	|   1 	|    0 	|    0 	|    0 	|     1 	|
|     well 	|       0 	|   0 	|    1 	|        0 	|  1 	|   0 	|    0 	|    0 	|    1 	|     1 	|
|     ends 	|       0 	|   0 	|    1 	|        0 	|  0 	|   0 	|    0 	|    1 	|    0 	|     0 	|
|    <END> 	|       0 	|   0 	|    0 	|        0 	|  0 	|   0 	|    1 	|    1 	|    0 	|     0 	|

The rows (or columns) of this matrix provide one type of word vectors (those based on word-word co-occurrence), but the vectors will be large in general (linear in the number of distinct words in a corpus). Thus, our next step is to run dimensionality reduction. In particular, we will run *SVD* (Singular Value Decomposition), which is a kind of generalized *PCA* (Principal Components Analysis) to select the top  k  principal components.

Reducing the dimensionality of such vectors doesn't interterfere with the semantic relationship between words. Hence, *movie* will still be closer to *theater* than to *airplane*.

### Identify distinct words

Define a function that will return a list of unique words of the corpus as well as its size.

In [ ]:
def distinct_words(corpus):
    """ Determine a list of distinct words for the corpus.
        Params:
            corpus (list of list of strings): corpus of documents
        Return:
            corpus_words (list of strings): list of distinct words across the corpus, sorted (using python 'sorted' function)
            num_corpus_words (integer): number of distinct words across the corpus
    """
    corpus_words = []
    num_corpus_words = -1

    # ------------------

    # Flatten corpus and get distinct words
    corpus_words = sorted(set(word for doc in corpus for word in doc))

    # Number of distinct words
    num_corpus_words = len(corpus_words)

    # ------------------

    return corpus_words, num_corpus_words

In [ ]:
corpus_words, num_corpus_words = distinct_words(corpus)

print(corpus_words[:10])     # first 10 words alphabetically
print(num_corpus_words)      # vocabulary size

['0', '000', '04', '075', '1', '10', '110', '123', '125', '134']
1214


### Compute the co-occurence matrix

Write a method that constructs a co-occurrence matrix for a certain window-size  $n$  (with a default of $4$), considering words  $n$  before and  $n$  after the word in the center of the window.

In [ ]:
def compute_co_occurrence_matrix(corpus, window_size=4):
    """ Compute co-occurrence matrix for the given corpus and window_size (default of 4).

        Note: Each word in a document should be at the center of a window. Words near edges will have a smaller
              number of co-occurring words.

              For example, if we take the document "<START> All that glitters is not gold <END>" with window size of 4,
              "All" will co-occur with "<START>", "that", "glitters", "is", and "not".

        Params:
            corpus (list of list of strings): corpus of documents
            window_size (int): size of context window
        Return:
            M (a symmetric numpy matrix of shape (number of unique words in the corpus , number of unique words in the corpus)):
                Co-occurence matrix of word counts.
                The ordering of the words in the rows/columns should be the same as the ordering of the words given by the distinct_words function.
            word2Ind (dict): dictionary that maps word to index (i.e. row/column number) for matrix M.
    """
    words, num_words = distinct_words(corpus)
    M = None
    word2Ind = {}

    # ------------------

    # Build word to index mapping
    word2Ind = {word: i for i, word in enumerate(words)}

    # Initialize co-occurrence matrix
    M = np.zeros((num_words, num_words))

    # Populate co-occurrence counts
    for doc in corpus:
        for i, word in enumerate(doc):
            word_idx = word2Ind[word]

            # Context window boundaries
            start = max(0, i - window_size)
            end = min(len(doc), i + window_size + 1)

            for j in range(start, end):
                if i != j:
                    context_word = doc[j]
                    context_idx = word2Ind[context_word]

                    # Symmetric update
                    M[word_idx, context_idx] += 1
                    M[context_idx, word_idx] += 1

    # ------------------

    return M, word2Ind

* The compute co-occurrence matrix is an implementation of the word context information recording how common words occur within a corpus with respect to each other within a fixed context window.

* It takes two inputs: the corpus (a list of tokenized documents) and a window size (default 4), which defines how many words before and after a center word are considered as context.
The first step involves the original extraction of all the distinct words in the corpus, which is followed by the size of the vocabulary numwords.
A mapping of the words to the index is then developed by the use of enumerate(words), which will enable words to be represented in terms of index.

* The co-occurrence matrix M is initialized by zeros with using the np.zeros((numwords, numwords)) function in which rows and columns are linked to vocabulary words.

* The iteration through the documents in the corpus is a consequence of the fact that co-occurrence is only calculated within documents and documents are processed separately.

* A word in document is considered the center word and index of the word is retrieved by using word2Ind.

* The context window boundaries are computed with start = max(0, i - window_size) and end = min(len(doc), i + window_size + 1) to avoid going outside the document.

* The function then loops over all words in this window, skips the center word itself (if i != j), and retrieves the index of each neighboring word.

* Co-occurrence counts are updated symmetrically using M[word_idx, context_idx] += 1 and M[context_idx, word_idx] += 1, ensuring the matrix remains symmetric.

* The function gives out the final co-occurrence matrix M, that is the number of pairs of words, and word2Ind which is needed to know what the rows and columns mean in the matrix.

In [ ]:
M, word2Ind = compute_co_occurrence_matrix(corpus, window_size=2)

print(M.shape)
print(len(word2Ind))
print(np.allclose(M, M.T))


(1214, 1214)
1214
True


### Reduce the dimensionality of the co-occurence matrix

Construct a method that performs dimensionality reduction on the matrix to produce $k$-dimensional embeddings. Use *SVD* to take the top $k$ components and produce a new matrix of $k$-dimensional embeddings.

In our case, we will set $k=2$.

In [ ]:
def reduce_to_k_dim(M, k=2):
    """ Reduce a co-occurence count matrix of dimensionality (num_corpus_words, num_corpus_words)
        to a matrix of dimensionality (num_corpus_words, k) using the following SVD function from Scikit-Learn:
            - http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html

        Params:
            M (numpy matrix of shape (number of unique words in the corpus , number of unique words in the corpus)): co-occurence matrix of word counts
            k (int): embedding size of each word after dimension reduction
        Return:
            M_reduced (numpy matrix of shape (number of corpus words, k)): matrix of k-dimensioal word embeddings.
                    In terms of the SVD from math class, this actually returns U * S
    """
    n_iters = 10     # Use this parameter in your call to `TruncatedSVD`
    M_reduced = None
    print("Running Truncated SVD over %i words..." % (M.shape[0]))

    # ------------------
    # Reduce the dimensionality of the co-occurrence matrix using SVD. Each word is now represented by a k-dimensional vector.
    svd = TruncatedSVD(n_components=k, n_iter=n_iters, random_state=0)
    M_reduced = svd.fit_transform(M)

    # ------------------

    print("Done.")
    return M_reduced

In [ ]:
M_reduced_co_occurrence = reduce_to_k_dim(M, k=2)
print(M_reduced_co_occurrence.shape)

Running Truncated SVD over 1214 words...
Done.
(1214, 2)


---

Great! You now have fix-sized vectors that represent each words of your corpus. Let's normalize our matrix to compare our vectors easily.

In [ ]:
# Rescale (normalize) the rows to make them each of unit-length
M_lengths = np.linalg.norm(M_reduced_co_occurrence, axis=1)
M_normalized = M_reduced_co_occurrence / M_lengths[:, np.newaxis] # broadcasting

Since we are working with vectors, we can easily measure the similarity between them using the dot product. Hence, given a specific word and its related word embedding, we can easily identify its most similar words contained in the corpus!

*Note: the dot product between two vectors is bounded between -1 and 1 and the dot product of two identical vectors is equal to 1.*

Define a function that given a word $w$ identify its most similar words in your embedding space.

In [ ]:
def most_similar(word, matrix, topn=10):
    """Return the words that have the closest embedding to the queried word."""

    # Get index of the query word
    word_idx = word2Ind[word]

    # Get embedding of the query word
    word_vec = matrix[word_idx]

    # Compute cosine similarity (dot product since vectors are normalized)
    similarities = matrix.dot(word_vec)

    # Sort words by similarity (descending)
    ranked_indices = np.argsort(-similarities)

    # Convert indices to words, excluding the query word itself
    ranked_words = [ind2word[i] for i in ranked_indices if ind2word[i] != word]

    return ranked_words[:topn]


## GloVe encoding

Word2Vec models are predictive by essence, since it is a neural network. However, this is not the sole method to learn geometrical encodings (vectors) of words from their co-occurrence information (how frequently they appear together in large text corpora).

GloVe is a count-based model that learn their vectors by essentially doing dimensionality reduction on the co-occurrence counts matrix. Does it remind you of something? Yes, that's exactly what you did above.

Building models is time consuming. Hence, GloVe / Word2Vec models already trained on regular training sets (Wikipedia, News, etc.) are publicly shared to be reused easily.

The below code will load a Glove model trained on wikipedia and allow us inspect easily the embeddings properties.

In [ ]:
import gensim.downloader as api

In [ ]:
# List all pretrained models available on gensim*
pp(list(api.info()["models"].keys()))

['fasttext-wiki-news-subwords-300',
 'conceptnet-numberbatch-17-06-300',
 'word2vec-ruscorpora-300',
 'word2vec-google-news-300',
 'glove-wiki-gigaword-50',
 'glove-wiki-gigaword-100',
 'glove-wiki-gigaword-200',
 'glove-wiki-gigaword-300',
 'glove-twitter-25',
 'glove-twitter-50',
 'glove-twitter-100',
 'glove-twitter-200',
 '__testing_word2vec-matrix-synopsis']


In [ ]:
model = api.load('glove-wiki-gigaword-50')

You can get the most similar embeddings to those of a given set of words.
Here, we retrieve the most similar words to fox, rabbit and cat.

In [ ]:
pp(model.most_similar(positive=["fox", "rabbit", "cat"]), compact=True)

[('dog', 0.8569531440734863), ('mouse', 0.7859790921211243),
 ('monster', 0.7710846066474915), ('wolf', 0.7690606713294983),
 ('bunny', 0.765525221824646), ('spider', 0.7395666241645813),
 ('duck', 0.7366620898246765), ('rat', 0.7366542220115662),
 ('beast', 0.7319128513336182), ('cartoon', 0.724099338054657)]


You can also perform concept additions / soustractions. For instance, you can

In [ ]:
model.most_similar(positive=["king", "woman"], negative=["man"])

[('queen', 0.8523604273796082),
 ('throne', 0.7664334177970886),
 ('prince', 0.7592144012451172),
 ('daughter', 0.7473883628845215),
 ('elizabeth', 0.7460219860076904),
 ('princess', 0.7424570322036743),
 ('kingdom', 0.7337412238121033),
 ('monarch', 0.721449077129364),
 ('eldest', 0.7184861898422241),
 ('widow', 0.7099431157112122)]

Explore those embeddings and comment on how they help to identify / handle:

* Synonyms
* Antonyms
* Grammatical errors
* Polysemy
* Irony
* Analogies

* Synonyms Embeddings combine synonyms closely since words that have similar meanings are likely to be used in similar situations.

Ex: Such words as car, automobile, and vehicle are frequently used in nearly the same phrases (buy a car, drive a vehicle, own an automobile). As the model reencounters them again and again it consequently learns that they are related in meaning and positions them near each other in the embedding space.  

* Antonyms: The similarity in context can be illustrated by the appearance of antonyms in similar contexts, and embeddings can place them next to each other, even though they have opposite meanings.

Ex: Both such words as hot and cold appear in weather-related sentences (a hot day, a cold day). In the model perspective, the words surrounding it appear very close thus it is very hard to differentiate similar meanings when it is only used in a context.  

* Grammatical errors: Embeddings are indirectly able to reflect unusual or wrong use of language, but are not very good with grammar because they are dedicated to semantics and not syntax.

Ex: The sentence such as She go to work every day will still contain the sentence She goes to work every day that will be very close in the embedding space. Although the grammar is incorrect, the meaning is unmistakable, and embeddings largely convey that meaning as opposed to the grammatical form.  

* Polysemy: Polysemy defines an average of all the senses of a single word and as such, it becomes hard to differentiate among the various senses.

Ex: a bank may refer to a bank or to a riverbank. Traditional embeddings have one vector per word, meaning that the two meanings are combined. This may be confusing in situations where the context matters to be used in understanding which meaning is meant.  

* Irony: Irony is contextual and pragmatic, which cannot be effectively represented in co-occurrence-based embeddings.

Ex: If someone says "Great job!" Once humans commit an error, they realize the irony. Embedding model, however, will primarily perceive great and job as positive words and will probably read the sentence as positive, without the slightest trace of sarcasticism in it.  

* Analogies: There are analogical relationships which arise due to regular co-occurrence patterns which produce linear structures in the embedding space.

Ex: king - man + woman = queen

This is true since the language does not alter its similarity in its association of gender to royalty. With time the model acquires these patterns and indicates them in the form of directions in the vector space.

---

Where do you think those biases come from?

The training data is embedded with biases. Models get to learn actual text and thus, they inherit the preexisting patterns, stereotypes, and imbalances of the corpus.

# Part II - Sentence representations

## TF-IDF

The TF-IDF is a methodology aiming at finding the most significative words in each document by comparing their in-document frequency to the overall frequency of that term in the whole corpus.

---

Convert the reuters corpus (at least one category) to its TF-IDF representation using [scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html). Beware of all the parameters of the methods! They might have significant impact on what you're doing.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Convert tokenized corpus into text documents
texts = [" ".join(doc) for doc in corpus]

# TF-IDF vectorization
vectorizer = TfidfVectorizer(
    lowercase=True,
    stop_words='english',
    max_df=0.95,
    min_df=2
)

tfidf_matrix = vectorizer.fit_transform(texts)


* Since TfidfVectorizer from scikit-learn expects raw text, the corpus is first converted back into text documents
* Each document becomes a single string
* The TfidfVectorizer is then initialized with several important parameters:
* lowercase=True: ensures all words are lowercased for consistency
* stop_words='english': removes common English stop words that carry little semantic meaning
* max_df=0.95: ignores words that appear in more than 95% of documents (too frequent to be informative)
* min_df=2: ignores words that appear in fewer than 2 documents (too rare)

Once you have converted your corpus using the TF-IDF methodology, create a function identifying the most relevant comments given a search query.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def search_corpus(corpus, search_query, topn=10):
    """Retrieve the top n documents matching a search query."""

    # Transform query using the SAME vectorizer
    query_vec = vectorizer.transform([search_query])

    # Compute cosine similarity
    similarities = cosine_similarity(query_vec, tfidf_matrix).flatten()

    # Rank documents
    top_indices = similarities.argsort()[::-1][:topn]

    return [(corpus[i], similarities[i]) for i in top_indices]

* The search_corpus function retrieves the most relevant documents for a given search query using TF-IDF vectors.
* The search query is transformed into a TF-IDF vector using the same vectorizer
*	Cosine similarity is then computed between the query vector and all document vectors
* Documents are ranked by similarity score
* The function returns: the most relevant documents from the corpus, with their similarity scores


In [ ]:
results = search_corpus(corpus, "tea export prices", topn=5)

for doc, score in results:
    print("Score:", score)
    print(doc[:30])
    print("-"*50)

Score: 0.2705409548378635
['<START>', 'india', 'relaxes', 'rules', 'for', 'export', 'promotion', 'the', 'reserve', 'bank', 'of', 'india', 'rbi', 'announced', 'new', 'rules', 'to', 'allow', 'exporters', 'of', '25', 'products', 'to', 'use', 'foreign', 'exchange', 'up', 'to', '10', 'pct']
--------------------------------------------------
Score: 0.17652327932492534
['<START>', 'vietnam', 'to', 'resettle', '300', '000', 'on', 'state', 'farms', 'in', '1987', 'vietnam', 'will', 'resettle', '300', '000', 'people', 'on', 'state', 'farms', 'known', 'as', 'new', 'economic', 'zones', 'in', '1987', 'to', 'create', 'jobs']
--------------------------------------------------
Score: 0.08543426723167645
['<START>', 'indonesian', 'agriculture', 'growth', 'expected', 'to', 'slow', 'indonesia', 's', 'agriculture', 'sector', 'will', 'grow', 'by', 'just', '1', '0', 'pct', 'in', 'calendar', '1987', 'against', 'an', 'estimated', '2', '4', 'pct', 'in', '1986', 'as']
--------------------------------------------

## Unsupervised Random Walk Sentence Embeddings

This approach has been presented by Kawin Ethayarajh in 2018. The key idea behind this methodology is to take a weighted average of previously trained word embeddings and modify it with SVD (Singular Value Decomposition, a kind of generalization of the PCA).

By having a look at [this implementation](https://github.com/kawine/usif), try to compute the uSIF embeddings of our corpus and compare their properties to the TF-IDF / Averaged Word2Vec ones.

Consider digging in the [related paper](https://aclanthology.org/W18-3012.pdf) if you want to know more about the methodology.

Unsupervised Random Walk Sentence Embeddings (uSIF)

uSIF is an unsupervised method proposed by Kawin Ethayarajh (2018) to build sentence embeddings from pre-trained word vectors. Instead of simply averaging word embeddings, uSIF assigns lower weights to very frequent words and higher weights to more informative ones.

The main idea is that frequent words (such as “the”, “is”, or “and”) appear in almost every sentence and tend to add noise rather than meaning. After computing a weighted average of word embeddings for each sentence, uSIF applies SVD and removes the most dominant component, which often corresponds to this shared, non-informative information.

Compared to simple averaged Word2Vec, uSIF produces sentence vectors that are less dominated by common words and better capture semantic differences between sentences. Compared to TF-IDF, uSIF keeps semantic information from pre-trained embeddings while still reducing the impact of frequent terms.

Overall, uSIF offers a good trade-off between simplicity and performance: it remains lightweight and unsupervised, while producing sentence embeddings that are more robust and semantically meaningful than basic averaging or TF-IDF representations.

# Extra : Latent Dirichlet Allocation (LDA)

Using the scikit-learn implementation of [LDA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html), trained on the "train" part of dataset:

What words do you find inside the main topics describing the first 5 articles in "housing", "money-fx" and "coffee"?

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
# We focus on these 3 categories as requested
categories = ["housing", "money-fx", "coffee"]

# Dictionary that will store the documents for each category
docs_by_category = {}

for cat in categories:
    # Get the first 5 article IDs for the category
    fileids = reuters.fileids(cat)[:5]

    # Convert each article into a single lowercase text string
    docs = [" ".join(reuters.words(fid)).lower() for fid in fileids]

    # Store the documents by category
    docs_by_category[cat] = docs

In [ ]:
# CountVectorizer turns text into a matrix of word counts. We remove common stopwords to focus on informative words
vectorizer = CountVectorizer(
    stop_words="english",
    max_df=0.95,   # ignore extremely frequent words
    min_df=2       # keep words that appear at least twice
)

# Stores the vectorized documents for each category
vectors_by_category = {}

for cat, docs in docs_by_category.items():
    # Transforms the documents into a word count matrix
    X = vectorizer.fit_transform(docs)

    # Saves the matrix for later use with LDA
    vectors_by_category[cat] = X

In [ ]:
# This function returns the top words of a topic based on their weights
def display_topics(topic, feature_names, top_n=10):
    # Sort words by importance and keep the top ones
    return feature_names.argsort()[-top_n:][::-1]

In [ ]:
for cat, X in vectors_by_category.items():
    # We use one topic since we only want the main theme per category
    lda = LatentDirichletAllocation(
        n_components=1,
        random_state=0)

    # Trains the LDA model on the word count matrix
    lda.fit(X)

    # Retrieves the vocabulary words
    feature_names = np.array(vectorizer.get_feature_names_out())

    # Extracts the most important words of the topic and displays the results
    topic_words = display_topics(lda.components_[0], feature_names)

    print(f"\nCategory: {cat}")
    print("Top topic words:", topic_words)

In summary, LDA on the first five articles of each category were applied to obtain coherent and intuitive topics.

In the case of the **housing** category, the words covering the dominant part of the topic are associated with real estate and financing, including the words mortgage, loan, rates, home, and market. These words are a clear indication of discussion on the prices of housing, lending rates as well as property market.

In the case of **money-fx**, the most dominant subject has such words as dollar, yen, currency, exchange and rates. This is in line with the foreign exchange news which is concerned with the currency movements and international financial markets.

In the case of **coffee**, some of the words extracted are coffee, beans, export, crop and prices. These words are closely connected with the production of agriculture, trade, and the price of commodities.

However, generally, a small amount of documents can be successfully analyzed through LDA to identify meaningful topics that align with the semantic content of every category.